In [1]:
# Install packages not available in the docker image
!pip install psycopg2-binary==2.9.3
!pip install -U pandasql

In [2]:
import os
from sqlalchemy import create_engine
import pandas as pd

# Create connection based on environment variables defined at docker-compose.yml
conn_url = "postgresql+psycopg2://%s:%s@db:5432/%s" % (
    os.environ["POSTGRES_USER"],
    os.environ["POSTGRES_PASSWORD"],
    os.environ["POSTGRES_DB"]
)
engine = create_engine(conn_url, isolation_level="AUTOCOMMIT")

### Show available datasets

In [3]:
from IPython.display import Markdown, display

dfs = {}
with engine.begin() as connection:
    for tbl in ["buyers", "merchants", "orders_creation", "orders_decision", "orders_payment", "datasets"]:
        df = pd.read_sql(f"SELECT * FROM {tbl}", connection)
        index_col = next(c for c in df if c.endswith("_id"))
        dfs[tbl] = df.set_index(index_col)
        
        display(Markdown(f"**{tbl}**"))
        display(dfs[tbl].head(n=10))

**buyers**

,first_name,last_name,email,created_at
buyer_id,,,,
1,Felicitas,Dowerg,weinhagekarl-heinz@example.org,2019-01-05 16:03:48.065322
2,Hellmuth,Etzold,stiffeljohanne@example.net,2019-01-06 02:18:23.014051
3,Dieter,Scheel,bbeckmann@example.org,2019-01-06 18:14:18.912525
4,Heinz-Jürgen,Heintze,rosehans-uwe@example.net,2019-01-07 23:18:11.857542
5,Friedlinde,Trapp,chartmann@example.net,2019-01-08 10:44:52.550876
6,Karl-Hans,Albers,hildaloeffler@example.com,2019-01-08 17:48:44.117045
7,Karl-Hans,Förster,vanessawilms@example.com,2019-01-10 16:08:24.980765
8,Edmund,Davids,wdietz@example.com,2019-01-12 15:39:50.044459
9,Dörte,Hettner,wagnervalerij@example.net,2019-01-13 09:43:09.301140


**merchants**

,name,created_at
merchant_id,,
1,Dowerg,2019-01-01 18:36:21.374000
2,Weinhage GbR,2019-01-30 18:17:05.389742
3,Hauffer AG & Co. KG,2019-03-08 06:13:25.480632
4,Dobes,2019-03-10 12:02:26.723510
5,Weinhage,2019-03-15 18:53:56.577648
6,Christoph,2019-04-11 11:14:38.032554
7,Löffler,2019-04-23 09:05:19.099020
8,Becker AG,2019-04-24 14:47:02.835260
9,Heintze GmbH & Co. OHG,2019-06-23 06:27:02.565517


**orders_creation**

...

,decision,due_date,created_at
order_id,,,
1,ACCEPT,2019-02-04 16:03:48.248094,2019-01-05 16:03:48.248094
2,ACCEPT,2019-02-05 02:18:23.093798,2019-01-06 02:18:23.093798
3,ACCEPT,2019-02-05 18:14:19.026141,2019-01-06 18:14:19.026141
4,ACCEPT,2019-02-06 23:18:11.898825,2019-01-07 23:18:11.898825
5,ACCEPT,2019-02-07 10:44:52.943724,2019-01-08 10:44:52.943724
6,ACCEPT,2019-02-07 17:48:44.200912,2019-01-08 17:48:44.200912
7,ACCEPT,2019-02-08 00:17:37.363524,2019-01-09 00:17:37.363524
8,ACCEPT,2019-02-09 16:08:25.024673,2019-01-10 16:08:25.024673
9,ACCEPT,2019-02-09 17:15:11.205784,2019-01-10 17:15:11.205784


**orders_payment**

,created_at
order_id,
1,2019-02-10 08:16:48.757377
2,2019-02-07 10:43:44.381343
3,2019-02-01 04:58:48.207862
4,2019-02-07 12:06:58.226781
5,2019-02-06 03:41:38.824075
6,2019-02-11 02:17:47.622723
7,2019-02-07 19:46:51.983749
8,2019-02-08 19:50:00.076555
9,2019-02-04 17:07:21.860173


**datasets**

,dataset_type
order_id,
1,train
2,train
3,train
4,train
5,train
6,train
7,train
8,train
9,train


The following steps were implemented to successfully complete this task

In [4]:
# Consolidated data
with engine.begin() as connection:
    raw_df = pd.read_sql(f""" SELECT
   oc.order_id,
   oc.buyer_id,
   b.first_name,
   b.last_name,
   b.email,
   m.name,
   m.created_at merchant_date,
   b.created_at buyers_date,
   oc.merchant_id,
   total_price_cents,
   oc.created_at orders_date,
   od.decision,
   od.due_date,
   od.created_at decision_date,
   op.created_at payment_date,
   d.dataset_type 
FROM
   orders_creation oc 
   LEFT JOIN
      buyers b 
      ON b.buyer_id = oc.buyer_id 
   LEFT JOIN
      merchants m 
      ON m.merchant_id = oc.merchant_id 
   LEFT JOIN
      orders_decision od 
      ON od.order_id = oc.order_id 
   LEFT JOIN
      orders_payment op 
      ON op.order_id = oc.order_id 
   LEFT JOIN
      datasets d 
      ON d.order_id = oc.order_id 
    """, connection)

In [5]:
# Train set
with engine.begin() as connection:
    train = pd.read_sql(f""" SELECT
   oc.order_id,
   oc.buyer_id,
   b.first_name,
   b.last_name,
   b.email,
   m.name,
   m.created_at merchant_date,
   b.created_at buyers_date,
   oc.merchant_id,
   total_price_cents,
   oc.created_at orders_date,
   od.decision,
   od.due_date,
   od.created_at decision_date,
   op.created_at payment_date,
   d.dataset_type 
FROM
   orders_creation oc 
   LEFT JOIN
      buyers b 
      ON b.buyer_id = oc.buyer_id 
   LEFT JOIN
      merchants m 
      ON m.merchant_id = oc.merchant_id 
   LEFT JOIN
      orders_decision od 
      ON od.order_id = oc.order_id 
   LEFT JOIN
      orders_payment op 
      ON op.order_id = oc.order_id 
   LEFT JOIN
      datasets d 
      ON d.order_id = oc.order_id 
WHERE
   d.dataset_type = 'train'
    """, connection)

In [8]:
# Test set
with engine.begin() as connection:
    test = pd.read_sql(f""" SELECT
   oc.order_id,
   oc.buyer_id,
   b.first_name,
   b.last_name,
   b.email,
   m.name,
   m.created_at merchant_date,
   b.created_at buyers_date,
   oc.merchant_id,
   total_price_cents,
   oc.created_at orders_date,
   od.decision,
   od.due_date,
   od.created_at decision_date,
   op.created_at payment_date,
   d.dataset_type 
FROM
   orders_creation oc 
   LEFT JOIN
      buyers b 
      ON b.buyer_id = oc.buyer_id 
   LEFT JOIN
      merchants m 
      ON m.merchant_id = oc.merchant_id 
   LEFT JOIN
      orders_decision od 
      ON od.order_id = oc.order_id 
   LEFT JOIN
      orders_payment op 
      ON op.order_id = oc.order_id 
   LEFT JOIN
      datasets d 
      ON d.order_id = oc.order_id 
WHERE
   d.dataset_type = 'test'
    """, connection)

## 2. Create your Target

In order to create the target variable and explore other meaninful features. The train set was explored 

In [ ]:
from pandasql import sqldf
psql = lambda q: sqldf(q, globals()) 

In [ ]:
query = '''
select
   BUYER_ID,
   COUNT(*) 
from
   train
where
   decision = 'ACCEPT' 
GROUP BY
   BUYER_ID 
HAVING
   COUNT(*) > 1 
ORDER BY
   1
'''

psql(query)

In [9]:
test.shape

(1258, 16)

In [6]:
train.to_sql('train_set', con=engine, if_exists='replace',index=False)

89

In [10]:
test.to_sql('test_set', con=engine, if_exists='replace',index=False)

258

In [ ]:
final.to_sql('test_set', con=engine, if_exists='replace',index=False)